In [43]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

**Imports**
---



In [44]:
import numpy as np
import pandas as pd
from pathlib import Path
import sys
import os
import glob
import multiprocessing as mp


pd.options.mode.chained_assignment = None  # default='warn'

In [45]:
p = Path('/Users/alex/Dropbox/FFE')

edge_file = p / 'FinnShapeEdges.parquet'
wind_file = p / 'Copy of GD_wind.csv'
folder = p / 'output'

In [46]:
# load dataaaa
wind_data = pd.read_csv(wind_file) 
edgelist = pd.read_parquet(edge_file, engine='pyarrow')

In [47]:
rngFile = edgelist[['source', 'IgnProbBld']]
rngFile.drop_duplicates(inplace=True)

**Definitions & parallel computing**
---

In [48]:
# num_cores = multiprocessing.cpu_count()
# print(num_cores)
# # need to do this on Shell / Terminal:
# conda install ipyparallel
# ipcluster nbextension enable --user
# ipcluster start # or ipcluster start -n 4

In [49]:
import ipyparallel as ipp
client = ipp.Client()
dview = client[:]
client.ids

[0, 1, 2, 3]

In [50]:
# add variables to all engines
dview["edgelist"]=edgelist
dview["rngFile"]=rngFile
dview["wind_data"]=wind_data
dview["folder"]=folder

# add all libraries to engines
with dview.sync_imports():
    import numpy as np
    import pandas as pd
    from pathlib import Path
    import sys
    import os
    import glob

importing numpy on engine(s)
importing pandas on engine(s)
importing Path from pathlib on engine(s)
importing sys on engine(s)
importing os on engine(s)
importing glob on engine(s)


KeyboardInterrupt: 

In [ ]:
%%px

def wind_scenario(wind_data):
      import numpy as np
      i = np.random.randint(0, wind_data.values.shape[0])
      w = wind_data.values[i, 2]
      dist = wind_data.values[i, 1]
      b = wind_data.values[i, 3]
      bear_max = b + 45  # wind direction
      bear_min = b - 45
      if b == 360:
          bear_max = 45
      if b <= 0:  # should not be necessary
          bear_min = 0
      if b == 999:
          bear_max = 999
          bear_min = 0
      return bear_max, bear_min, dist # wind characteristics, bearing and distance


def ignition(rngList=rngFile, edges=edgelist):
    rngList['rng'] = np.random.uniform(0, 1, size=rngList.values.shape[0])
    rngList = rngList[rngList['rng'] < rngList['IgnProbBld']]
    NewActiveEdges = edges[edges['source'].isin(rngList['source'])]
    return NewActiveEdges


def mask(t, activeEdges_d, listActivatedSources_d, w_b_max, w_b_min, w_d):
    import numpy as np
    if t==0: # special case at time=0
        return activeEdges_d
    else:
        mask = (activeEdges_d.bearing.values < w_b_max) & (activeEdges_d.bearing.values < w_b_min) & (activeEdges_d.distance < w_d)
        NewActiveEdges = activeEdges_d[mask]
        NewActiveEdges = NewActiveEdges[~NewActiveEdges.source.isin(listActivatedSources_d)]
        return NewActiveEdges


def propagation(activeEdges_d, edges=edgelist):
    import numpy as np
    import pandas as pd
    NewActiveEdges = edges[edges.source.isin(activeEdges_d.target)]
    return NewActiveEdges


In [ ]:
@dview.parallel(block = False) # The @parallel decorator breaks up elementwise operations and distributes them.
def ffe_runs(n):
    import numpy as np
    import pandas as pd
    for scenario in n:
        # initial setup
        listActivatedSources = []
        listScenarioDataframes = []
        condition = True
        time = 0 
        # wind conditions
        w_bearing_max, w_bearing_min, w_distance = wind_scenario(wind_data)
        # ignition / initial state and edges selection
        ActiveEdges = ignition()
        # print(f"{len(ActiveEdges)} ignitions")
        if ActiveEdges.empty:
            continue
        while condition: # spread burn zone
            ActiveEdges = mask(time, ActiveEdges, listActivatedSources, w_bearing_max, w_bearing_min, w_distance)
            if ActiveEdges.empty: #no more buildings to burn
                break
            listScenarioDataframes.append(ActiveEdges)
            listActivatedSources.extend(ActiveEdges.source.values)
            ActiveEdges = propagation(ActiveEdges)
            time += 1
        
        print(f'finishing pid {os.getpid()} scenario --- {scenario} time ---- {time}')

        Activations = pd.concat(listScenarioDataframes)
        Activations["scenario"] = scenario
        Activations["pid"] = os.getpid()
        Activations.to_parquet(str(folder) + '/' + f'scenario{scenario}_pid{os.getpid()}.parquet', engine='auto', compression="GZIP")
        

**Main**
---


---



In [ ]:
%%time
ffe_runs(range(300))


**Backup**
---



---



In [51]:
def clean_up(path=folder):
    files = path.glob('*scenario*')
    for f in files:
        print(f)
        os.remove(f)

In [52]:
clean_up()

/Users/alex/Dropbox/FFE/output/scenario234_pid40089.parquet
/Users/alex/Dropbox/FFE/output/scenario203_pid40087.parquet
/Users/alex/Dropbox/FFE/output/scenario58_pid40083.parquet
/Users/alex/Dropbox/FFE/output/scenario184_pid40087.parquet
/Users/alex/Dropbox/FFE/output/scenario122_pid40086.parquet
/Users/alex/Dropbox/FFE/output/scenario162_pid40087.parquet
/Users/alex/Dropbox/FFE/output/scenario251_pid40089.parquet
/Users/alex/Dropbox/FFE/output/scenario75_pid40086.parquet
/Users/alex/Dropbox/FFE/output/scenario93_pid40086.parquet
/Users/alex/Dropbox/FFE/output/scenario22_pid40083.parquet
/Users/alex/Dropbox/FFE/output/scenario103_pid40086.parquet
/Users/alex/Dropbox/FFE/output/scenario270_pid40089.parquet
/Users/alex/Dropbox/FFE/output/scenario6_pid40083.parquet
/Users/alex/Dropbox/FFE/output/scenario47_pid40083.parquet
/Users/alex/Dropbox/FFE/output/scenario264_pid40089.parquet
/Users/alex/Dropbox/FFE/output/scenario157_pid40087.parquet
/Users/alex/Dropbox/FFE/output/scenario169_pid4

/Users/alex/Dropbox/FFE/output/scenario160_pid40087.parquet
/Users/alex/Dropbox/FFE/output/scenario253_pid40089.parquet
/Users/alex/Dropbox/FFE/output/scenario186_pid40087.parquet
/Users/alex/Dropbox/FFE/output/scenario120_pid40086.parquet
/Users/alex/Dropbox/FFE/output/scenario64_pid40083.parquet
/Users/alex/Dropbox/FFE/output/scenario201_pid40087.parquet
/Users/alex/Dropbox/FFE/output/scenario236_pid40089.parquet
/Users/alex/Dropbox/FFE/output/scenario164_pid40087.parquet
/Users/alex/Dropbox/FFE/output/scenario257_pid40089.parquet
/Users/alex/Dropbox/FFE/output/scenario269_pid40089.parquet
/Users/alex/Dropbox/FFE/output/scenario124_pid40086.parquet
/Users/alex/Dropbox/FFE/output/scenario182_pid40087.parquet
/Users/alex/Dropbox/FFE/output/scenario232_pid40089.parquet
/Users/alex/Dropbox/FFE/output/scenario141_pid40086.parquet
/Users/alex/Dropbox/FFE/output/scenario60_pid40083.parquet
/Users/alex/Dropbox/FFE/output/scenario0_pid40083.parquet
/Users/alex/Dropbox/FFE/output/scenario41_pi